## FLEX, Bad-Beads WHBL vs PBMC

In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(H5weaver)
library(hise)
library(tidyverse)
library(SeuratObject)
library(SeuratDisk)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: Matrix

Loading required package: rhdf5


Attaching package: ‘H5weaver’


The following objects are masked from ‘package:rhdf5’:

    h5dump, h5ls


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.3     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ 

### Reading in the H5s

In [2]:
h5s_fix <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d', 
    pattern = 'filtered_feature_bc_matrix.h5$', 
    full.names = TRUE, 
    recursive = TRUE
)


In [3]:
h5s_fix

[1] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05012_Bead_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[2] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05012_FACS_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[3] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05044_Bead_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[4] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05044_FACS_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[5] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05045_Bead_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[6] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05045_FACS_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[7] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/PB04745_Bead_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"
[8] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/PB04745_FACS_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"

In [4]:
h5s_fix[1]

[1] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05012_Bead_Fix_neg_RNase2_sample_filtered_feature_bc_matrix.h5"

In [5]:
scrub <- list.files(
    # Place your analysis in a folder by number experiment, or whatever might work for you
    # It helps if this worksheet in in the same folder
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d',
    pattern = '.csv$', # change this to 'sample_raw_feature_bc_matrix.h5$' if you need the raw h5s
    full.names = TRUE, 
    recursive = TRUE
)
scrub

[1] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05012_Bead_Fix_neg_RNase2.csv"
[2] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05012_FACS_Fix_neg_RNase2.csv"
[3] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05044_Bead_Fix_neg_RNase2.csv"
[4] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05044_FACS_Fix_neg_RNase2.csv"
[5] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05045_Bead_Fix_neg_RNase2.csv"
[6] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/BL05045_FACS_Fix_neg_RNase2.csv"
[7] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/PB04745_Bead_Fix_neg_RNase2.csv"
[8] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/scrublet_fig2d/PB04745_FACS_Fix_neg_RNase2.csv"

In [6]:
scrubs <- lapply(scrub,read.csv)

scrubs <- scrubs %>% lapply(function(x){
    rownames(x) <- x$X
    x$X <- NULL
    x$Barcodes <- rownames(x)
    return(x)
})

In [7]:
fully <- lapply(h5s_fix, function(h5){
    # x <- h5[[1]]

    l <- length(strsplit(h5 ,'/')[[1]])
    pro <- strsplit(h5 ,'/')[[1]][l]
    pro <- strsplit(pro, '_RNase')[[1]][1]
    mtx <- Read10X_h5(h5)
    so <- CreateSeuratObject(mtx,project=pro)
    return(so)
    })


In [8]:
for (i in c(1:length(fully))){
    
    fully[[i]][[]]$Barcodes <- rownames(fully[[i]][[]])
    
    fully[[i]][[]] <- left_join(fully[[i]][[]], scrubs[[i]], by = 'Barcodes') # , by='row.names', all=TRUE
}

In [9]:
fully <- Reduce(merge, fully) %>% JoinLayers()


In [10]:
fully

An object of class Seurat 
18082 features across 84899 samples within 1 assay 
Active assay: RNA (18082 features, 0 variable features)
 1 layer present: counts

In [11]:
head(fully)

,orig.ident,nCount_RNA,nFeature_RNA,Barcodes,Doublet_Score,Predicted_Doublet
,<chr>,<dbl>,<int>,<chr>,<dbl>,<chr>
AAACAAGCAAAGCCTAACTTTAGG-1,BL05012_Bead_Fix_neg,909,740,AAACAAGCAAAGCCTAACTTTAGG-1,0.10989011,False
AAACAAGCACCACACCACTTTAGG-1,BL05012_Bead_Fix_neg,1148,872,AAACAAGCACCACACCACTTTAGG-1,0.19183673,False
AAACAAGCATAATGACACTTTAGG-1,BL05012_Bead_Fix_neg,714,611,AAACAAGCATAATGACACTTTAGG-1,0.03522205,False
AAACAAGCATAGACCAACTTTAGG-1,BL05012_Bead_Fix_neg,1581,1180,AAACAAGCATAGACCAACTTTAGG-1,0.14285714,False
AAACCAATCACATTACACTTTAGG-1,BL05012_Bead_Fix_neg,1658,1270,AAACCAATCACATTACACTTTAGG-1,0.07540984,False
AAACCAATCCAACTTGACTTTAGG-1,BL05012_Bead_Fix_neg,1740,1282,AAACCAATCCAACTTGACTTTAGG-1,0.20084567,False
AAACCAATCGTTGCACACTTTAGG-1,BL05012_Bead_Fix_neg,2684,1600,AAACCAATCGTTGCACACTTTAGG-1,0.10236220,False
AAACCAGGTCACGATTACTTTAGG-1,BL05012_Bead_Fix_neg,1311,1057,AAACCAGGTCACGATTACTTTAGG-1,0.25773196,False
AAACCAGGTCCATCAGACTTTAGG-1,BL05012_Bead_Fix_neg,1003,766,AAACCAGGTCCATCAGACTTTAGG-1,0.02072539,False


## MT - filtering

In [12]:
# The [[ operator can add columns to object metadata. This is a great place to stash QC stats
fully[["percent.mt"]] <- PercentageFeatureSet(fully, pattern = "^MT-")
fully <- subset(fully, subset = percent.mt < 5)

In [13]:
fully <- subset(fully, Predicted_Doublet == 'False')

In [14]:
fully$run <- 'fix'
fully$orig.ident <- substr(fully$orig.ident,0,16)

fully$orig.ident<-substr(start=0 ,stop=16,fully$orig.ident)

fully$clean<-substr(start=9 ,stop=12,fully$orig.ident)

fully$type<-substr(start=1 ,stop=2,fully$orig.ident)

fully$type <- gsub('BL','Whole Blood', fully$type)
fully$type <- gsub('PB','Leukopak', fully$type)

fully$sample <- substr(start=0,stop=7,fully$orig.ident)

fully$sample2 <- fully$sample
fully$sample2 <- gsub('BL05012','WHBL-1', fully$sample2)
fully$sample2 <- gsub('BL05044','WHBL-2', fully$sample2)
fully$sample2 <- gsub('BL05045','WHBL-3', fully$sample2)
fully$sample2 <- gsub('PB04745','Leukopak', fully$sample2)


In [15]:
fully <- NormalizeData(fully) %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>% 
    RunUMAP(dims = 1:20) %>% 
    FindNeighbors(dims = 1:20) %>% 
    FindClusters(resolution = 0.5)

Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  IL7R, FCMR, TRBC1, LTB, CD5, PYHIN1, SAMD3, CCL5, IL2RB, CTSW 
	   PRF1, KLRG1, KLRK1, NKG7, LEF1, PIM2, ITGB7, GZMA, TRIB2, CST7 
	   NELL2, CD7, CD8A, CCR7, CD28, OBSCN, GATA3, MATK, MYBL1, CD27 
Negative:  SPI1, CST3, TNFAIP2, PLXNB2, NCF2, TYMP, EMILIN2, SERPINA1, IFI30, LYZ 
	   CLEC7A, LRP1, CSF3R, LILRB2, MPEG1, HK3, HCK, CD68, CFP, CYBB 
	   S100A9, MEFV, KCTD12, MNDA, VCAN, KLF4, ZNF385A, CD300E, LRRC25, PRAM1 
PC_ 2 
Positive:  CD79A, NIBAN3, IGHM, MS4A1, IGHD, CD22, PAX5, FCRL1, FCRL2, BLK 
	   BANK1, TNFRSF13C, IGKC, POU2AF1, CD79B, CD19, RALGPS2, WDFY4, OSBPL10, AFF3 
	   FCER2, FCRLA, IGHG3, SWAP70, BCL11A, CXCR5, RUBCNL, ADAM28, CDK14, BLNK 
Negative:  PRF1, NKG7, CST7, CCL5, SYNE1, GZMA, ADGRG1, CTSW, GNLY, IL2RB 
	   KLRD1, SAMD3, SRGN, MATK, FGFBP2, FCRL6, GZMH, KLRK1, CD300A, PLEKHG3 
	   CX3CR1, S1PR5, TBX21, KLRG1, ANXA1, FOSL2, MYBL1, PRR5L, 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 79792
Number of edges: 2732795

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9319
Number of communities: 23
Elapsed time: 47 seconds


In [16]:
fully

An object of class Seurat 
18082 features across 79792 samples within 1 assay 
Active assay: RNA (18082 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 2 dimensional reductions calculated: pca, umap

### Celltype label transfer

In [17]:
ref <- LoadH5Seurat(file = '/home/jupyter/pbmc_multimodal.h5seurat')


Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding variable feature information for ADT

Adding miscellaneous information for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for w

In [18]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = fully,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
)


Normalizing query using reference SCT model

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 8710 anchors



In [19]:
fully <- TransferData(
    anchorset = anchors, 
    reference = ref, 
    query = fully,
    refdata = list(
        celltype.l1 = "celltype.l1",
        celltype.l2 = "celltype.l2",
        celltype.l3 = "celltype.l3",
        predicted_ADT = 'ADT'
    )
)


Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Transfering 228 features onto reference data

Warning message:
“Layer counts isn't present in the assay object; returning NULL”


### Plotting and saving 

In [20]:
saveRDS(fully, '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/02_Data/2D_exp747/Fig2D_Final.rds')

In [21]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] SeuratDisk_0.0.0.9020 lubridate_1.9.3       forcats_1.0.0        
 [4] stringr_1.5.1         purrr_1.0.2           readr_2.1.5          
 [7] tidyr_1.3.1           tibble_3.2.1          ggplot2_3.4.3        
[10] tidyverse_2.0.0       hise_2.15.0           H5weaver_1.2.0       
[13] rhdf5_2.38.1          M